In [ ]:
library(dplyr)
neg <- read.table("/data/yaoq2/TTseq/20231027/neg_active_name.bdg", header = FALSE, sep = '\t')
pos <- read.table("/data/yaoq2/TTseq/20231027/pos_active_name.bdg", header = FALSE, sep = '\t')
colnames(neg) <- c('Geneid')
colnames(pos) <- c('Geneid')
nrow(neg)
nrow(pos)


In [ ]:
Top1_fpkm <- read.table("/data/stringtie_Top1_gene_coverage_all_samples.tsv", header = TRUE, sep = '\t')


Top1_fpkm$NAA1_norm <- Top1_fpkm$Top1_NAA1/0.349491239612692
Top1_fpkm$NAA2_norm <- Top1_fpkm$Top1_NAA2/0.522744198947793
Top1_fpkm$IAA1_norm <- Top1_fpkm$Top1_IAA1/2.87008170834299
Top1_fpkm$IAA2_norm <- Top1_fpkm$Top1_IAA2/1.99470515239475

#Top1_fpkm$NAA2_norm <- Top1_fpkm$Top1.NAA2
#Top1_fpkm$IAA1_norm <- Top1_fpkm$Top1.IAA1
#Top1_fpkm$IAA2_norm <- Top1_fpkm$Top1.IAA2

head(Top1_fpkm)

z1 <- select(Top1_fpkm, "Gene_ID", "NAA1_norm")
z2 <- select(Top1_fpkm, "Gene_ID", "NAA2_norm")
z3 <- select(Top1_fpkm, "Gene_ID", "IAA1_norm")
z4 <- select(Top1_fpkm, "Gene_ID", "IAA2_norm")

z1['name'] = "AUX"
z2['name'] = "AUX"
z3['name'] = "IAA"
z4['name'] = "IAA"

colnames(z1) <- c('Geneid', 'TPM', 'name')
colnames(z2) <- c('Geneid', 'TPM', 'name')
colnames(z3) <- c('Geneid', 'TPM', 'name')
colnames(z4) <- c('Geneid', 'TPM', 'name')


z1_neg <- semi_join(z1, neg, by = "Geneid")
z2_neg <- semi_join(z2, neg, by = "Geneid")
z1_pos <- semi_join(z1, pos, by = "Geneid")
z2_pos <- semi_join(z2, pos, by = "Geneid")
z3_neg <- semi_join(z3, neg, by = "Geneid")
z4_neg <- semi_join(z4, neg, by = "Geneid")
z3_pos <- semi_join(z3, pos, by = "Geneid")
z4_pos <- semi_join(z4, pos, by = "Geneid")

df_neg1 <- bind_rows(z1_neg, z3_neg)
df_neg2 <- bind_rows(z2_neg, z4_neg)
df_neg1['group'] <- "repeat1"
df_neg2['group'] <- "repeat2"
df_neg <- bind_rows(df_neg1, df_neg2)

df_pos1 <- bind_rows(z1_pos, z3_pos)
df_pos2 <- bind_rows(z2_pos, z4_pos)
df_pos1['group'] <- "repeat1"
df_pos2['group'] <- "repeat2"
df_pos <- bind_rows(df_pos1, df_pos2)


df_neg2['SD'] = "Neg"
df_pos2['SD'] = "Pos"
df <- bind_rows(df_neg2, df_pos2)
#head(df)
df_neg1['SD'] = "Neg"
df_pos1['SD'] = "Pos"
df1 <- bind_rows(df_neg1, df_pos1)

In [ ]:
cols <- c("#2367AC", "#8DBEE7")
p1 <- ggplot(df_neg, aes(x= name, y= log2(TPM), fill = group))+
  #geom_violin(trim = F) +
  geom_boxplot() +
  ggtitle("HCT116-Top1 \n Genes in negative domains") +
  labs(y = "log2(Coverage)")+
  theme_classic() +
  ylim(-5, 10) +
  scale_x_discrete(limits=c("AUX", "IAA"))+ 
  scale_fill_manual(values=cols)+
  theme(legend.position="none", axis.title.x = element_blank(), plot.title = element_text(hjust = 0.5))
  
p2 <- ggplot(df_pos, aes(x= name, y= log2(TPM), fill = group))+
  #geom_violin(trim = F) +
  geom_boxplot() +
  ggtitle("HCT116-Top1 \n Genes in positive domains") +
  labs(y = "log2(Coverage)")+
  theme_classic() +
  ylim(-6, 10) +
  scale_x_discrete(limits=c("AUX", "IAA"))+ 
  scale_fill_manual(values=cols)+
  #theme(legend.position= c(0.8, 0.85), axis.title.x = element_blank())
  theme(legend.position= "right", axis.title.x = element_blank(), plot.title = element_text(hjust = 0.5))
  
p = p1 + p2
p
#ggsave(plot = p, width = 7, height = 3, dpi = 300, filename = "figure/*.pdf")

In [ ]:
library(ggpubr)
compare_means(TPM ~ name, data = df_neg1)
compare_means(TPM ~ name, data = df_pos1)
compare_means(TPM ~ name, data = df_neg2)
compare_means(TPM ~ name, data = df_pos2)